In [95]:
import pysolr
import requests
import json
from pprint import pprint
from IPython.display import display, display_html

In [96]:
# Solr collection url 
SOLR_URL = 'http://solr:8983/solr/cmp269'

SOLR_SCHEMA_URL = "%s/%s" % (SOLR_URL, 'schema') 

FIELD_TYPES_LIST_URL = "%s/%s" % (SOLR_SCHEMA_URL, 'fieldtypes')
DYNAMIC_FIELDS_LIST_URL = "%s/%s" % (SOLR_SCHEMA_URL, 'dynamicfields')

## Define algumas funções para criar tipos e campos no Solr, via API
As funções abaixo são usadas para obter a lista de _tipos de campos_ e de _campos dinâmicos_ atualmente existentes no schema da coleção e também para criar tipos quando não existirem. Sempre que um novo tipo é criado, se não há um campo dinâmico para aquele tipo o campo também é criado.

In [97]:
def get_current_types():
    response = requests.get(FIELD_TYPES_LIST_URL)
    if response.status_code != requests.codes.ok:
        raise Exception("Falha ao identificar os tipos de campos existentes no schema")

    current_field_types = response.json()["fieldTypes"]
    
    return current_field_types

def get_current_fields():
    response = requests.get(DYNAMIC_FIELDS_LIST_URL)
    if response.status_code != requests.codes.ok:
        raise Exception("Falha ao identificar os tipos dinâmicos existentes no schema")

    current_fields = response.json()["dynamicFields"]
    
    return current_fields

def create_dynamic_field_for_type(ftype_name):
    
    sufix = ftype_name.split("_")[-1]
    fname = "*_txt_%s" % sufix
    
    current_fields = get_current_fields()
    for field in current_fields:
        if field["name"] == fname:
            print("Campo '%s' já existe. Ignorando.." % fname)
            print("")
            print("Definição atual do campo é:")
            pprint(field)
            print("")
            return False
    
    
    field_def =  {
        "add-dynamic-field":{
            "name": fname,
            "type": ftype_name,
            "stored": True 
        }  
    }
    
    field_def_json = json.dumps(field_def)

    headers = { 
        "Content-type": "application/json"
    }
    
    response = requests.post(SOLR_SCHEMA_URL, data=field_def_json, headers=headers)
    
    
    return response.status_code == requests.codes.ok
    
    
def create_field_type(type_def):
    """Cria um campo cuja definção é descrita no argumento 'type_def'
        
        O argumento é 'inspecionado' para se obter o nome do tipo, então a 
        existência do tipo é verificada antes da criação.
        
        Se o tipo for criado, a função que cria campos dinâmicos é invocada.
    """
    current_field_types = get_current_types()
    
    ftype_name = type_def["add-field-type"]["name"]
    sufix = ftype_name.split("_")[-1]
    ftype = "text_%s" % sufix
    
    for ftype in current_field_types:
        if ftype["name"] == ftype_name:
            print("Tipo %s já existe. Ignorando.." % ftype_name)
            print("")
            print("Definição atual do tipo é:")
            pprint(ftype)
            print("")
            return create_dynamic_field_for_type(ftype_name)
        
    type_def_json = json.dumps(type_def)
    headers = { 
        "Content-type": "application/json"
    }

    # Creates the field type
    response = requests.post(SOLR_SCHEMA_URL, data=type_def_json, headers=headers)
       
    if response.status_code == requests.codes.ok:
        return create_dynamic_field_for_type(ftype_name)
    else:
        raise Exception(response.text)


## Cria o campo tipo de campo text_c1 (c1 é de 'Customizado 1'...)
Esse campo apenas vai usar o [Standard Tokenizer](https://lucene.apache.org/solr/guide/6_6/tokenizers.html#Tokenizers-StandardTokenizer)

In [98]:
# Custom Field Type 1
field_type_def1 = {
    "add-field-type" : {
        "name": "text_c1",
        "class": "solr.TextField",
        "positionIncrementGap": "100",
        "analyzer" : {
            "tokenizer": {
                "class": "solr.StandardTokenizerFactory"
            },
            "filters": []
        }
    }
}

create_field_type(field_type_def1)


Tipo text_c1 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c1',
 'positionIncrementGap': '100'}

Campo '*_txt_c1' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c1', 'stored': True, 'type': 'text_c1'}



False

## Cria o tipo de campo text_c2.
Além do Standard Tokenizer, ele usará o [Lower Case Filter](https://lucene.apache.org/solr/guide/7_2/filter-descriptions.html#FilterDescriptions-LowerCaseFilter).

In [99]:
# Copies the previous field
field_type_def2 = field_type_def1.copy()

# Changes what should be different
field_type_def2["add-field-type"]["name"] = "text_c2"
field_type_def2["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.LowerCaseFilterFactory"
})

# creates the new type and field
create_field_type(field_type_def2)

Tipo text_c2 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c2',
 'positionIncrementGap': '100'}

Campo '*_txt_c2' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c2', 'stored': True, 'type': 'text_c2'}



False

## Cria o tipo de campo text_c3
Esse tipo além do Standard Tokenizer e do Lower Case Filter, faz remoção de stop words, no idioma portugês do Brasil, usando [StopFilterFactory](https://lucene.apache.org/solr/guide/7_2/filter-descriptions.html#stop-filter)

In [100]:
field_type_def3 = field_type_def2.copy()

field_type_def3["add-field-type"]["name"] = "text_c3"
field_type_def3["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.StopFilterFactory",
    "format": "snowball",
    "words": "lang/stopwords_pt.txt",
    "ignoreCase": True
})

create_field_type(field_type_def3)

Tipo text_c3 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'},
                          {'class': 'solr.StopFilterFactory',
                           'format': 'snowball',
                           'ignoreCase': 'true',
                           'words': 'lang/stopwords_pt.txt'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c3',
 'positionIncrementGap': '100'}

Campo '*_txt_c3' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c3', 'stored': True, 'type': 'text_c3'}



False

## Cria o tipo de campo text_c4
Tem tudo que havia nos customizados anteriores e mais o Steam, usando __PortugueseLightSteamFilter__.
As configurações deste tipo de campo __equivalem a do tipo _text_pt_ __ que é builtin no Solr e usado para os campos dinâmicos do cujo padrão de nomes é _*_txt_pt_

In [101]:
field_type_def4 = field_type_def3.copy()

field_type_def4["add-field-type"]["name"] = "text_c4"
field_type_def4["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.PortugueseLightStemFilterFactory"
})

create_field_type(field_type_def4)

Tipo text_c4 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'},
                          {'class': 'solr.StopFilterFactory',
                           'format': 'snowball',
                           'ignoreCase': 'true',
                           'words': 'lang/stopwords_pt.txt'},
                          {'class': 'solr.PortugueseLightStemFilterFactory'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c4',
 'positionIncrementGap': '100'}

Campo '*_txt_c4' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c4', 'stored': True, 'type': 'text_c4'}



False

## Cria o tipo text_c5 
Esse campo é como o _text_c4_, porém usa um steammer diferente, o __PortugueseMinimalStemFilterFactory__. que é registrado nos comentários do tipo "text\_pt" (no arquivo managed-schema) como sendo __less aggressive__. Creio que em relação ao _PortugueseLightSteamFilter_.

In [102]:
field_type_def5 = field_type_def3.copy()  # copy field 3 again

field_type_def5["add-field-type"]["name"] = "text_c5"
field_type_def5["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.PortugueseMinimalStemFilterFactory"
})

create_field_type(field_type_def5)

Tipo text_c5 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'},
                          {'class': 'solr.StopFilterFactory',
                           'format': 'snowball',
                           'ignoreCase': 'true',
                           'words': 'lang/stopwords_pt.txt'},
                          {'class': 'solr.PortugueseLightStemFilterFactory'},
                          {'class': 'solr.PortugueseMinimalStemFilterFactory'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c5',
 'positionIncrementGap': '100'}

Campo '*_txt_c5' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c5', 'stored': True, 'type': 'text_c5'}



False

## Cria o tipo text_c6 
Esse campo é como o _text_c3_, porém usa um steammer diferente, o [SnowballPorterFilterFactory](http://snowball.tartarus.org/algorithms/portuguese/stemmer.html). Este steammer é gerado pelo software _snowball_ e é do tipo _pattern steammer_, que é supostamente menos preciso que um _table steammer_, mas é também mais simples de manter e mais rápido..[documentação do Solr](https://lucene.apache.org/solr/guide/7_1/filter-descriptions.html#snowball-porter-stemmer-filter)


In [103]:
field_type_def6 = field_type_def3.copy()  # copy field 3 again

field_type_def6["add-field-type"]["name"] = "text_c6"
field_type_def6["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.SnowballPorterFilterFactory",
    "language": "Portuguese"
})

create_field_type(field_type_def6)

Tipo text_c6 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'},
                          {'class': 'solr.StopFilterFactory',
                           'format': 'snowball',
                           'ignoreCase': 'true',
                           'words': 'lang/stopwords_pt.txt'},
                          {'class': 'solr.PortugueseLightStemFilterFactory'},
                          {'class': 'solr.PortugueseMinimalStemFilterFactory'},
                          {'class': 'solr.SnowballPorterFilterFactory',
                           'language': 'Portuguese'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c6',
 'positionIncrementGap': '100'}

Campo '*_txt_c6' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c6', 'stored': True, 'type': 'text_c6'}



False

## Cria o tipo text_c7 
Esse campo é como o _text_c3_, mas usando o __PortugueseStemFilterFactory__ descrito nos comentários do arquivo de configuração (managed-schema) do Solr como o __most agressive__.

In [104]:
field_type_def7 = field_type_def3.copy()  # copy field 3 again

field_type_def7["add-field-type"]["name"] = "text_c7"
field_type_def7["add-field-type"]["analyzer"]["filters"].append({
    "class": "solr.PortugueseStemFilterFactory"
})

create_field_type(field_type_def7)

Tipo text_c7 já existe. Ignorando..

Definição atual do tipo é:
{'analyzer': {'filters': [{'class': 'solr.LowerCaseFilterFactory'},
                          {'class': 'solr.StopFilterFactory',
                           'format': 'snowball',
                           'ignoreCase': 'true',
                           'words': 'lang/stopwords_pt.txt'},
                          {'class': 'solr.PortugueseLightStemFilterFactory'},
                          {'class': 'solr.PortugueseMinimalStemFilterFactory'},
                          {'class': 'solr.SnowballPorterFilterFactory',
                           'language': 'Portuguese'},
                          {'class': 'solr.PortugueseStemFilterFactory'}],
              'tokenizer': {'class': 'solr.StandardTokenizerFactory'}},
 'class': 'solr.TextField',
 'name': 'text_c7',
 'positionIncrementGap': '100'}

Campo '*_txt_c7' já existe. Ignorando..

Definição atual do campo é:
{'name': '*_txt_c7', 'stored': True, 'type': 'text_c7'}



False